In [1]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [35]:
recipe = pd.read_csv('./data/recipe.csv')
recipe = recipe[['recipe_id', 'recipe_name']]

## start

In [ ]:
ratings = pd.read_csv('./data/ratings.csv')
ratings = ratings[['recipe_id', 'user_id', 'rating']]
ratings = ratings.sort_values(by=['recipe_id'], axis=0)

In [36]:
## new test
ratings = pd.read_csv('./data/new_ratings.csv')
ratings = ratings[['recipe_id', 'user_id', 'rating']]
ratings = ratings.sort_values(by=['recipe_id'], axis=0)

In [37]:
users = pd.read_csv('./data/users.csv')

In [38]:
ratings = pd.merge(ratings, recipe, on=['recipe_id'], how='outer')
ratings = ratings.sort_values(by=['recipe_id'], axis=0)

In [39]:
ratings = pd.merge(ratings, users, on='user_id', how='outer')
ratings = ratings.sort_values(by=['user_id'], axis=0)

In [ ]:
# ratings = ratings.dropna(subset=['recipe_id'])

In [40]:
user_num = ratings.user_id.unique().shape[0]
recipe_num = ratings.recipe_id.unique().shape[0]

print("유저의 수는 {}명 이고, \n레시피의 수는 {}개 입니다.".format(user_num, recipe_num))

유저의 수는 101명 이고, 
레시피의 수는 1309개 입니다.


In [41]:
# 사용자 샘플 선출 -> pivot table 생성
UM_matrix_ds = ratings.pivot(index='user_id', columns='recipe_id', values='rating')
UM_matrix_ds.head(5)

recipe_id,NaN,recipe-1,recipe-10,recipe-100,recipe-1000,recipe-1001,recipe-1002,recipe-1003,recipe-1004,recipe-1005,...,recipe-990,recipe-991,recipe-992,recipe-993,recipe-994,recipe-995,recipe-996,recipe-997,recipe-998,recipe-999
user_id,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user-1,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,4.0,NaN,4.0,2.0,NaN,0.0,NaN
user-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user-11,NaN,2.0,1.0,4.0,2.0,0.0,NaN,2.0,NaN,NaN,...,4.0,NaN,0.0,4.0,NaN,4.0,1.0,0.0,1.0,0.0


In [9]:
def distance_cosine(a, b) :
       return 1-distance.cosine(a, b)

In [10]:
def distance_correlation(a, b) :
       return 1-distance.correlation(a, b)

In [11]:
def distance_euclidean(a, b) :
       return 1/(1+distance.euclidean(a, b))

In [12]:
def nearest_neighbor_user(user, topN, simFunc) :
    u1  =  UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = { }
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = [ ]
        interSectionU2 = [ ]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim):nn[uid] = sim
                                                
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [43]:
# for user in range(0, len(UM_matrix_ds)) :
#       print('User {0} neighbors : {1}'.format('user-'+str(user+1), nearest_neighbor_user('user-'+str(user+1), 3, distance_cosine)))

for user in range(0, 20) :
      print('User {0} neighbors : {1}'.format('user-'+str(user+1), nearest_neighbor_user('user-'+str(user+1), 3, distance_cosine)))

User user-1 neighbors : [('user-8', 0.9819805060619657), ('user-5', 0.9729928210195775), ('user-4', 0.9528933774279027)]
User user-2 neighbors : [('user-8', 0.9721855188786469), ('user-13', 0.9437001160274026), ('user-92', 0.9428090415820632)]
User user-3 neighbors : [('user-8', 0.9500587703260617), ('user-87', 0.9344877349289681), ('user-84', 0.9344877349289681)]
User user-4 neighbors : [('user-5', 0.9753568020509494), ('user-8', 0.9716656205775166), ('user-7', 0.95291375803987)]
User user-5 neighbors : [('user-99', 1.0), ('user-8', 0.9916405353625916), ('user-97', 0.9827076298239908)]
User user-6 neighbors : [('user-5', 0.9560240446570316), ('user-8', 0.9438798074485388), ('user-1', 0.9320071091029151)]
User user-7 neighbors : [('user-5', 0.9748960603557335), ('user-4', 0.95291375803987), ('user-1', 0.9496285473385507)]
User user-8 neighbors : [('user-58', 1.0), ('user-9', 0.9917017107805809), ('user-5', 0.9916405353625916)]
User user-9 neighbors : [('user-8', 0.9917017107805809), ('

In [44]:
def predict_rating(userid, nn=20, simFunc=distance_cosine):
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_recipe = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh=1)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for recipe_id, row in neighbor_recipe.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([recipe_id, wsum/jsum])
        
    return ret

## user - recipe rating 예측 결과

# cosine

In [45]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
    predict = predict_rating(user_id, 20, distance_cosine)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf_cosine = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf_cosine

,user_id,recipe_id,rating
0,user-1,recipe-1,1.521314
1,user-1,recipe-10,0.678471
2,user-1,recipe-100,2.728504
3,user-1,recipe-1000,1.336405
4,user-1,recipe-1001,0.478686
...,...,...,...
92830,user-100,recipe-995,3.234249
92831,user-100,recipe-996,3.596033
92832,user-100,recipe-997,3.540628
92833,user-100,recipe-998,3.461173


In [46]:
predict_cosine = pd.merge(left = ratings[['recipe_id','user_id','rating']], right = resultdf_cosine, on=['user_id','recipe_id'])
predict_cosine

,recipe_id,user_id,rating_x,rating_y
0,recipe-592,user-1,1.0,1.548037
1,recipe-593,user-1,4.0,4.000000
2,recipe-590,user-1,4.0,3.392038
3,recipe-59,user-1,4.0,3.569317
4,recipe-589,user-1,4.0,3.732663
...,...,...,...,...
36272,recipe-602,user-99,4.0,3.532695
36273,recipe-63,user-99,4.0,3.735714
36274,recipe-642,user-99,4.0,3.737718
36275,recipe-511,user-99,4.0,3.591613


In [47]:
sample_all_cosine = pd.merge(left = ratings , right = resultdf_cosine, how = "outer", on = ["user_id", 'recipe_id'])
sample_all_cosine

,recipe_id,user_id,rating_x,recipe_name,rating_y
0,recipe-592,user-1,1.0,속채운대저토마토백김치,1.548037
1,recipe-593,user-1,4.0,둥지튀김,4.000000
2,recipe-590,user-1,4.0,탕평채,3.392038
3,recipe-59,user-1,4.0,된장 두부찌개,3.569317
4,recipe-589,user-1,4.0,귀리죽,3.732663
...,...,...,...,...,...
110952,recipe-995,user-100,NaN,NaN,3.234249
110953,recipe-996,user-100,NaN,NaN,3.596033
110954,recipe-997,user-100,NaN,NaN,3.540628
110955,recipe-998,user-100,NaN,NaN,3.461173


In [48]:
sample_cosine = sample_all_cosine[sample_all_cosine['recipe_id'] == 'recipe-1000']
sample_cosine = sample_cosine.sort_values(by='user_id' ,ascending=True)
sample_cosine.head(60)

,recipe_id,user_id,rating_x,recipe_name,rating_y
54401,recipe-1000,user-1,NaN,NaN,1.336405
61288,recipe-1000,user-10,NaN,NaN,2.052639
109745,recipe-1000,user-100,NaN,NaN,3.425578
1028,recipe-1000,user-11,2.0,닭가슴살호두크로켓,1.951670
1781,recipe-1000,user-12,1.0,닭가슴살호두크로켓,1.327489
63189,recipe-1000,user-13,NaN,NaN,2.427152
64257,recipe-1000,user-14,NaN,NaN,1.490263
3581,recipe-1000,user-15,1.0,닭가슴살호두크로켓,1.988191
65040,recipe-1000,user-16,NaN,NaN,0.504693
65708,recipe-1000,user-17,NaN,NaN,3.306486


In [49]:
sample_cosine = sample_cosine.fillna(value=0)
sample_cosine.tail()

,recipe_id,user_id,rating_x,recipe_name,rating_y
105951,recipe-1000,user-94,0.0,0,2.885336
107177,recipe-1000,user-95,0.0,0,4.000000
107341,recipe-1000,user-96,0.0,0,4.000000
107428,recipe-1000,user-97,0.0,0,3.501526
108581,recipe-1000,user-99,0.0,0,3.272158


# 오류지표

In [50]:
realdata_rating_cosine = sample_cosine['rating_x'].tolist()
resultdata_rating_cosine = sample_cosine['rating_y'].tolist()

error_rate_absol_cosine = mean_absolute_error(realdata_rating_cosine, resultdata_rating_cosine)
error_rate_squared_cosine = mean_squared_error(realdata_rating_cosine, resultdata_rating_cosine)

### MAE
##### 예측값-결과값 유사 평가 척도
MAE 값 오차의 정도에 관한 값으로 0에 가까울수록 유사도 높음

In [52]:
print("Error Rate(Absolute) : ", error_rate_absol_cosine)  # MAE

Error Rate(Absolute) :  1.8962523036011454


### MSE
##### 예측 정확성
MSE 값 적을수록 원본과 예측값의 차이 적어짐 -> 예측 정확성 높음

In [53]:
print("Error Rate(Squared) : ", error_rate_squared_cosine)  # MSE
# print("RMSE : ", math.sqrt(error_rate_squared))  # RMSE

Error Rate(Squared) :  5.743131558422857


# correlation

In [24]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 20, distance_correlation)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf_correlation = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf_correlation

/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,user_id,recipe_id,rating
0,user-1,recipe-1,1.589029
1,user-1,recipe-10,0.709923
2,user-1,recipe-100,2.649836
3,user-1,recipe-1000,1.371898
4,user-1,recipe-1001,0.410971
...,...,...,...
66372,user-54,recipe-995,3.426513
66373,user-54,recipe-996,1.777331
66374,user-54,recipe-997,2.052046
66375,user-54,recipe-998,1.943630


In [25]:
sample_all_correlation = pd.merge(left = ratings , right = resultdf_correlation, how = "outer", on = ["user_id", 'recipe_id'])
sample_all_correlation

,recipe_id,user_id,rating_x,recipe_name,rating_y
0,recipe-592,user-1,1.0,속채운대저토마토백김치,1.573499
1,recipe-593,user-1,4.0,둥지튀김,4.000000
2,recipe-590,user-1,4.0,탕평채,3.377701
3,recipe-59,user-1,4.0,된장 두부찌개,3.301616
4,recipe-589,user-1,4.0,귀리죽,3.687275
...,...,...,...,...,...
74176,recipe-936,user-54,NaN,NaN,2.036878
74177,recipe-961,user-54,NaN,NaN,2.677473
74178,recipe-967,user-54,NaN,NaN,2.773107
74179,recipe-977,user-54,NaN,NaN,1.000000


In [26]:
sample_correlation = sample_all_correlation[sample_all_correlation['recipe_id'] == 'recipe-1000']
sample_correlation = sample_correlation.sort_values(by='user_id' ,ascending=True)
sample_correlation.head(60)

,recipe_id,user_id,rating_x,recipe_name,rating_y
54401,recipe-1000,user-1,NaN,NaN,1.371898
61453,recipe-1000,user-10,NaN,NaN,2.377814
1028,recipe-1000,user-11,2.0,닭가슴살호두크로켓,0.838647
1781,recipe-1000,user-12,1.0,닭가슴살호두크로켓,1.458501
63389,recipe-1000,user-13,NaN,NaN,1.366918
64472,recipe-1000,user-14,NaN,NaN,1.285268
3581,recipe-1000,user-15,1.0,닭가슴살호두크로켓,1.308922
65353,recipe-1000,user-16,NaN,NaN,1.249508
66063,recipe-1000,user-17,NaN,NaN,1.171603
4993,recipe-1000,user-18,1.0,닭가슴살호두크로켓,1.125458


In [27]:
sample_correlation = sample_correlation.fillna(value=0)
sample_correlation.tail()

,recipe_id,user_id,rating_x,recipe_name,rating_y
52954,recipe-1000,user-86,4.0,닭가슴살호두크로켓,0.000000
52985,recipe-1000,user-88,4.0,닭가슴살호두크로켓,0.000000
53015,recipe-1000,user-89,4.0,닭가슴살호두크로켓,0.000000
60684,recipe-1000,user-9,0.0,0,1.397523
53514,recipe-1000,user-90,4.0,닭가슴살호두크로켓,0.000000


In [28]:
realdata_rating_correlation = sample_correlation['rating_x'].tolist()
resultdata_rating_correlation = sample_correlation['rating_y'].tolist()

error_rate_absol_correlation = mean_absolute_error(realdata_rating_correlation, resultdata_rating_correlation)
error_rate_squared_correlation = mean_squared_error(realdata_rating_correlation, resultdata_rating_correlation)

print("Error Rate(Absolute) : ", error_rate_absol_correlation)  # MAE
print("Error Rate(Squared) : ", error_rate_squared_correlation)  # MSE

Error Rate(Absolute) :  1.5128677571457616
Error Rate(Squared) :  4.073987090632987


# euclidean

In [30]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 20, distance_euclidean)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf_euclidean = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf_euclidean

,user_id,recipe_id,rating
0,user-1,recipe-10,4.000000
1,user-1,recipe-1000,4.000000
2,user-1,recipe-1010,4.000000
3,user-1,recipe-1023,4.000000
4,user-1,recipe-1044,4.000000
...,...,...,...
53951,user-100,recipe-995,3.788601
53952,user-100,recipe-996,4.000000
53953,user-100,recipe-997,4.000000
53954,user-100,recipe-998,4.000000


In [32]:
sample_all_euclidean = pd.merge(left = ratings , right = resultdf_euclidean, how = "outer", on = ["user_id", 'recipe_id'])
sample_euclidean = sample_all_euclidean[sample_all_euclidean['recipe_id'] == 'recipe-1000']
sample_euclidean = sample_euclidean.sort_values(by='user_id' ,ascending=True)
sample_euclidean.head(80)

,recipe_id,user_id,rating_x,recipe_name,rating_y
54400,recipe-1000,user-1,NaN,NaN,4.000000
57227,recipe-1000,user-10,NaN,NaN,1.738336
98591,recipe-1000,user-100,NaN,NaN,3.881415
1028,recipe-1000,user-11,2.0,닭가슴살호두크로켓,4.000000
1781,recipe-1000,user-12,1.0,닭가슴살호두크로켓,4.000000
...,...,...,...,...,...
83850,recipe-1000,user-84,NaN,NaN,4.000000
52916,recipe-1000,user-85,4.0,닭가슴살호두크로켓,4.000000
52954,recipe-1000,user-86,4.0,닭가슴살호두크로켓,4.000000
87464,recipe-1000,user-87,NaN,NaN,4.000000


In [33]:
sample_euclidean = sample_euclidean.fillna(value=0)
sample_euclidean.tail()

,recipe_id,user_id,rating_x,recipe_name,rating_y
94797,recipe-1000,user-94,0.0,0,3.230620
96023,recipe-1000,user-95,0.0,0,4.000000
96187,recipe-1000,user-96,0.0,0,4.000000
96274,recipe-1000,user-97,0.0,0,3.864300
97427,recipe-1000,user-99,0.0,0,3.617364


In [34]:
realdata_rating_euclidean = sample_euclidean['rating_x'].tolist()
resultdata_rating_euclidean = sample_euclidean['rating_y'].tolist()

error_rate_absol_euclidean = mean_absolute_error(realdata_rating_euclidean, resultdata_rating_euclidean)
error_rate_squared_euclidean = mean_squared_error(realdata_rating_euclidean, resultdata_rating_euclidean)

print("Error Rate(Absolute) : ", error_rate_absol_euclidean)  # MAE
print("Error Rate(Squared) : ", error_rate_squared_euclidean)  # MSE

Error Rate(Absolute) :  2.414632263292692
Error Rate(Squared) :  8.219110134846664
